# Mesa ABM of the Keep Threshold Relay

### Simulation notes and instructions

Notes:
* Nodes are created simulataneously using the Mesa SimultaneousActivation scheduler
* The nodes go through a bootstrap phase based on Antonio's diagram. They can only form groups if they are in the mainloop forked state.
* Relay requests are currently bernoulli trials
* Singature events are modeled by default as asynchronous with a step delay to mimick the block delay

Instructions:
1. Change variables
2. Click on the restart and run kernel icon for ipython

    

In [ ]:
import random
import simpy
import datetime
import numpy as np
import pandas as pd
import agent
import model
import matplotlib.pyplot as plt
import simulation_functions as sc

**Initialize Variables**

In [ ]:
group_size = 5 # Group size
#total_tickets = 500 # total virtual stakers
nodes = 5 # number of nodes
active_group_threshold = 4 # min number of active groups to pick from
signature_threshold = 4  # min number of nodes needed to sign
signature_delay = 2 # lambda for poisson distribution used to generate a delay before the signature is executed
group_expiry = 10 #number of steps after which a group expires
group_formation_threshold = 3 # min number of nodes needed to create a group
node_failure_percent = 50 # % of times nodes fail and have to reconnect
node_death_percent = 5 # % of time nodes fail and do not reconnect


In [ ]:
# Pre-processing Staker distributions
# Linear y = m * (x + 1) since indices start from 0
ticket_distribution = np.zeros(nodes) 

for i in range (0, nodes):
    ticket_distribution[i] =  10 * (i + 1) # m = 50 gives max y as 1000 which is 2% of 50000

x = np.linspace(1,nodes,nodes)
plt.subplot(2, 1, 1)
plt.bar(x, ticket_distribution)
plt.title("Virtual Staker Distribution")

cdf_L02=sc.create_cdf(nodes,ticket_distribution)
plt.subplot(2, 1, 2)
plt.bar(x, cdf_L02)
plt.title("Virtual Staker Cumulative Distribution")

In [ ]:
beacon_model = model.Beacon_Model(nodes, ticket_distribution, 
                                  active_group_threshold, 
                                  group_size, 
                                  signature_threshold, 
                                  group_expiry, 
                                  group_formation_threshold, 
                                  node_failure_percent, 
                                  node_death_percent, 
                                  signature_delay
                                 )

In [ ]:
active_group_size = []
signature_failures = []
for i in range (200):
    beacon_model.step()
    active_group_size.append(len(beacon_model.active_groups))
    print("signature failure events")
    print(beacon_model.unsuccessful_signature_events)
    

In [ ]:
plt.hist(active_group_size, bins=range(max(active_group_size)+1))
plt.xlabel("group size")
plt.ylabel("frequency")
print(np.median(active_group_size))

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(np.linspace(1,len(active_group_size),len(active_group_size)),active_group_size)
plt.xlabel("simulation time")
plt.ylabel("active groups")

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(np.linspace(1,len(beacon_model.unsuccessful_signature_events),
                     len(beacon_model.unsuccessful_signature_events)),
                     beacon_model.unsuccessful_signature_events)
plt.xlabel("simulation time")
plt.ylabel("Signature failure events")

In [ ]:
group_distr = beacon_model.datacollector.get_agent_vars_dataframe()

In [ ]:
print(group_distr)